#CHATBOT DE INFORMAÇÕES DE DOCENTES DA UFABC

O Chatbot proposto aqui é de responder dúvidas comuns como a especialidade, a formação ou os meios de contato de docentes da UFABC.
Os dados utilizados são retirados a partir da página da universidade, na qual é listado todos (ou a grande maioria) dos docentes que fazem parte dela: https://www.ufabc.edu.br/ensino/docentes/.

#Importação das bibliotecas necessárias

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
import numpy as np
import random
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
!pip install spacy --upgrade
import spacy
!python3 -m spacy download pt

#Funções utilizadas

Função que irá consultar na página de docentes da UFABC e retornar em texto o conteúdo encontrado caso exista essa pessoa.

-O nome digitado deve ser escrito sem acentos e em caso de 'ç', é preciso utilizar 'c'

In [ ]:
def encontrar_docente():
    nome = input('Por favor, digite o nome completo do docente desejado (desconsiderar acentos e troque "ç" por "c"): ')
    nome = nome.lower()
    nome = nome.replace(' ','-')

    try:
        dados = urllib.request.urlopen('https://www.ufabc.edu.br/ensino/docentes/'+nome)
        dados = dados.read()
        dados_html = bs.BeautifulSoup(dados, 'lxml')
        dados_html = dados_html.find_all(class_="item-page item-page")

        conteudo=''
        for p in dados_html:
            conteudo += p.text

        conteudo = conteudo.lower()
        conteudo = re.sub(r"\s{2,}",' ',conteudo)
        conteudo = re.sub(r"\:",'',conteudo)
        conteudo = re.sub(r"\n",' ',conteudo)
        return conteudo
    except:
        return ("Docente não encontrado")

Função que irá processar o texto, usando a lemmatização, tanto da pergunta do usuário como do que for encontrado na página.

In [ ]:
def preprocessamento(texto):
    pln = spacy.load('pt')
    stop_words = spacy.lang.pt.stop_words.STOP_WORDS
    # espacos em branco
    texto = re.sub(r" +", ' ', texto)

    documento = pln(texto)
    lista = []
    for token in documento:
        lista.append(token.lemma_)

    lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
    lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

    return lista

Função que irá analizar a melhor resposta possivel dado a pergunta, baseado na medida TF-IDF.

In [ ]:
def responder(texto_usuario,lista_sentencas):
    resposta_chatbot = ''
    lista_sentencas_preprocessada.append(texto_usuario)

    tfidf = TfidfVectorizer()
    palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_preprocessada)

    similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

    indice_sentenca = similaridade.argsort()[0][-2]
    vetor_similar = similaridade.flatten()
    vetor_similar.sort()
    vetor_encontrado = vetor_similar[-2]

    if (vetor_encontrado == 0):
        resposta_chatbot = resposta_chatbot + 'Desculpe, mas não entendi!'
        return resposta_chatbot
    else:
        resposta_chatbot = resposta_chatbot + lista_sentencas[indice_sentenca]
        return resposta_chatbot

Função que irá tokenizar o conteudo da página da web e fazer o processamento desses tokens

In [ ]:
def docprocess(conteudo):
    lista_sentencas = nltk.sent_tokenize(conteudo)
    for i in range(len(lista_sentencas)):
        lista_sentencas_preprocessada.append(preprocessamento(lista_sentencas[i]))
    return lista_sentencas

#Dialogo com o bot

Código principal do chatbot

In [ ]:
print('Olá, sou um chatbot e vou responder perguntas sobre os docentes da UFABC')
continuar = True
while (continuar == True):
    doc = encontrar_docente()

    if(doc!="Docente não encontrado"):
        lista_sentencas_preprocessada = []
        lista_sentencas = docprocess(doc)
        print("-------------------------------")
        print("Texto encontrado:")
        print(doc)
        print("-------------------------------")
        mesmo_doc = True

        while(mesmo_doc):
            print("O que deseja saber?")
            texto_usuario = input()
            texto_usuario = texto_usuario.lower()

            if (texto_usuario != 'encerrar' and texto_usuario != 'trocar docente'):
                print('Chatbot: ')
                print(responder(preprocessamento(texto_usuario),lista_sentencas))
                lista_sentencas_preprocessada.remove(preprocessamento(texto_usuario))

            elif (texto_usuario == 'trocar docente'):
                mesmo_doc = False
            
            else:
                continuar = False
                mesmo_doc = False
                print('Chatbot: Até breve!')

    else:
        print(doc)

Olá, sou um chatbot e vou responder perguntas sobre os docentes da UFABC
Por favor, digite o nome completo do docente desejado (desconsiderar acentos e troque "ç" por "c"): Francisco de assis zampirolli
-------------------------------
Texto encontrado:
 francisco de assis zampirolli área visão computacional; engenharia de software. possui graduação em matemática pela universidade federal do espírito santo (1992), mestrado em matemática aplicada pela universidade de são paulo (1997) e doutorado em engenharia elétrica pela universidade estadual de campinas (2003). tem experiência na área de ciência da computação, com ênfase em visão computacional (com aplicações em bioinformática, robótica, sensoriamento remoto, linhas de produção, etc) e engenharia de software (em padrões de software e geração automática de modelos e código). página pessoal contatos centro centro de matemática computação e cognição (cmcc) sala 819-b - bloco b - santo andré telefone +55 11 4996-0078 e-mail fzampirolli@uf

Nota-se que pode haver problemas dependendo de como está redigido o perfil de cada docente no site, pois não há uma padronização nessa parte.